In [1]:
#GLOBALS
import numpy as np
import general_robotics_toolbox as rox #ik
import time
import math
from Arm_Lib import Arm_Device #

# Create robot arm object
Arm = Arm_Device()
time.sleep(.1) #tenth of a second delay

#Defining variables
angle = 90
s_time = 1000 #milliseconds
gripper = 6

# Define the home configuration of the robot
H0 = np.eye(4)

In [2]:
def home(): #set arm to home position after task is complete. Useful for ik
    Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-90, s_time) #FIXME

In [3]:
################################################################################################################################################################################################################################################################
#FK Function 
################################################################################################################################################################################################################################################################
def forward_kinematics(joint_angles):
    return dofbot.fwdkin(joint_angles)

In [4]:
################################################################################################################################################################################################################################################################
#IK Function 
################################################################################################################################################################################################################################################################
# Robot parameters based on the URDF 
# link_lengths = {
#     'l0': 61 * 1e-3,
#     'l1': 43.5 * 1e-3,
#     'l2': 82.85 * 1e-3,
#     'l3': 82.85 * 1e-3,
#     'l4': 73.85 * 1e-3,
#     'l5': 54.57 * 1e-3
# }

# # Define the unit vectors for each axis
# ex = np.array([1, 0, 0])
# ey = np.array([0, 1, 0])
# ez = np.array([0, 0, 1])

# # Define the position vectors from joint i-1 to joint i
# p01 = -link_lengths['l0'] * ex + link_lengths['l1'] * ez
# p12 = link_lengths['l2'] * ez
# p23 = -link_lengths['l3'] * ex
# p34 = -link_lengths['l4'] * ex
# p45 = -link_lengths['l5'] * ey

# # IK function using the general robotics toolbox
# def calculate_ik(target_pose, joint_limits):
#     # Convert the target pose to a transformation matrix
#     target_transform = rox.Transform(rox.rpy2rot(target_pose[3:]), target_pose[:3])
    
#     # Calculate the inverse kinematics
#     q_ik = rox.invkin(dofbot, target_transform)

#     # Enforce joint limits
#     for i, (lower_limit, upper_limit) in enumerate(joint_limits):
#         q_ik[i] = max(min(q_ik[i], upper_limit), lower_limit) #radians 

#     joint_angles_degrees = [angle * 180.0 / np.pi for angle in q_ik] #degrees
    
#     return joint_angles_degrees

# # Define the joint limits based on your robot's specifications
# joint_limits = [
#     (-1.5708, 1.5708),  # joint1 limits
#     (-1.5708, 1.5708),  # joint2 limits
#     (-1.5708, 1.5708),  # joint3 limits
#     (-1.5708, 1.5708),  # joint4 limits
#     (-1.5708, 3.1416),  # joint5 limits
# ]

# import the needed modules (as is used to rename the module for easier referencing)
import math
import numpy as np
import general_robotics_toolbox as rox
from Arm_Lib import Arm_Device #

# define the Jacobian inverse 
def jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol):
# the inputs are 
	# robot: of the Robot class defined in rox. contains rotation axes, position vectors, joint types
	# q0: the initial guess joint angles as a 5x1 numpy array
	# Rd: the desired 3x3 rotation matrix (R_{0T}) as a numpy array
	# Pd: the desired 3x1 position vector (P_{0T}) as a numpy array
	# Nmax: the maximum number of allowed iterations
	# alpha: the update parameter
	# tol: the tolerances for [roll, pitch, yaw, x, y, z] as a 6x1 numpy array

# set up storage space
    n = len(q0)
    q = np.zeros((n,Nmax+1))
    q[:,0] = q0
    p0T = np.zeros((3,Nmax+1))
    RPY0T = np.zeros((3,Nmax+1))
    iternum = 0

# compute the forward kinematics
    H = rox.fwdkin(robot,q[:,0])
    R = H.R
    P = H.p
    P = np.array([[P[0]], [P[1]], [P[2]]])

# get the initial error
    dR = np.matmul(R, np.transpose(Rd))
    r = np.array(rox.R2rpy(dR))[None]
    dX = np.concatenate((np.transpose(r), P-Pd))

# iterate while any error element is greater than its tolerance
    while (np.absolute(dX) > tol).any():
	# stop execution if the maximum number of iterations is exceeded
        if iternum < Nmax:
		# compute the forward kinematics
            H = rox.fwdkin(robot, q[:,iternum])
            R = H.R
            p0T = H.p
            p0T = np.array([[p0T[0]], [p0T[1]], [p0T[2]]])
		# compute the error
            dR = np.matmul(R , np.transpose(Rd))
            r = np.array(rox.R2rpy(dR))[None]
            dX = np.concatenate((np.transpose(r), p0T-Pd))

		# calculate the Jacobian matrix
            Jq = rox.robotjacobian(robot, q[:, iternum])
		# compute the update
            j = np.matmul(np.linalg.pinv(Jq), dX)
		# use the update to generate a new q
            q[:, iternum+1] = q[:, iternum] - np.transpose((alpha * j))
            iternum = iternum + 1
        else:
            break
	# return the final estimate of q
    return q[:, iternum]

def main():

	#define inputs
	
    #qd=[30;45;80;25;40]
    #Rd = np.array([ [-0.75, -0.1047, -0.6531], [-0.433, 0.8241, 0.3652], [0.5, 0.5567, -0.6634]])
    #Pd = np.array([[0.2058], [0.1188], [0.1464]])
    #q0 = np.array(np.transpose([25, 50, 75, 30, 30]))*math.pi/180
    s_time = 1500 #milliseconds

    #qd=[0;45;135;45;135]
#     Rd = np.array([ [-0.7071, -0.5, -0.5], [0, -0.7071, 0.7071], [-0.7071, 0.5, 0.5]])
    Rd = np.array([ [0, -1, 0], [0, 0, 1], [-1, 0, 0]])

    Pd = np.array([[-0.21127], [0], [0.18735]])
	# convert initial guess to radians
    q0 = np.array(np.transpose([-5, 50, 130, 50, 130]))*math.pi/180
    
    tol = np.array(np.transpose([[0.02, 0.02, 0.02, 0.001, 0.001, 0.001]]))
    Nmax = 200
    alpha = 0.1

    # Define all the joint lengths [m]
    l0 = 61 * 10**-3
    l1 = 43.5 * 10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 54.57 * 10**-3

	# define the unit vectors
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    # Define the position vectors from i-1 -> i
    P01 = (l0 + l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ex
    P34 = -1*l3 * ez
    P45 = np.zeros(3)
    P5T = -1*(l4 + l5) * ex

	# define the class inputs: rotation axes (H), position vectors (P), and joint_type
    H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
    P = np.array([P01, P12, P23, P34, P45, P5T]).T
    joint_type = [0,0,0,0,0]

	# define the Robot class
    robot = rox.Robot(H, P, joint_type)
	
	# compute the inverse kinematics
    q = jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol)
	# convert solution to degrees
    q = q * 180 / math.pi
    print(q) #this gives an array of q values for the end effector. Could be simply inputted to servo commands like we used for fk. Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-90, s_time) #FIXME
    Arm.Arm_serial_servo_write6(q[0], q[1], q[2], q[3], q[4], 90, s_time) #FIXME
if __name__ == "__main__" :
    main()

[ 2.86705624e-12  1.74155670e+02  1.26032593e+02 -3.01882628e+01
  9.00000000e+01]


In [5]:
################################################################################################################################################################################################################################################################
#PATH FOLLOWING TASK
################################################################################################################################################################################################################################################################
#IMPLEMENTED FUNCTIONS


# Step 1: Generate a joint space path
def generate_joint_space_path(start_angles, end_angles, steps):
    path = [np.linspace(start, end, steps) for start, end in zip(start_angles, end_angles)]
    return np.transpose(path)

# Step 2: Command the arm to follow the path
def command_arm_to_follow_path(robot, path):
    for joint_angles in path:
        # This function should send the joint angles to your robot's actuators
        send_joint_angles_to_robot(joint_angles)
        time.sleep(0.1)  # Wait for some time interval before sending the next set of angles

# Step 3: Measure the actual position (this function should be replaced with actual sensor readout)
def measure_actual_position(robot):
    # This is a placeholder for actual sensor readout
    return get_joint_angles_from_robot()

# Step 4: Evaluate the error
def evaluate_path_following_error(desired_path, robot):
    error_path = []
    for desired_joint_angles in desired_path:
        actual_joint_angles = measure_actual_position(robot)
        error = desired_joint_angles - actual_joint_angles
        error_path.append(error)
    return error_path

In [6]:
################################################################################################################################################################################################################################################################
#FEEDBACK TASK
################################################################################################################################################################################################################################################################

In [7]:
# Example usage of the IK function with joint limits
target_pose = [0.2, 0.0, 0.1, 0, np.pi/2, 0]  # 
joint_angles = calculate_ik(target_pose, joint_limits)
print(f"The calculated joint angles for the target pose are: {joint_angles}")

NameError: name 'calculate_ik' is not defined

In [ ]:
# Example usage
# Define start and end joint angles and the number of steps in the path
start_angles = np.radians([0, -np.pi/4, np.pi/2, -np.pi/4, 0, np.pi/3])
end_angles = np.radians([np.pi/4, 0, -np.pi/2, np.pi/4, -np.pi/2, 0])
steps = 100

# Generate the path
path = generate_joint_space_path(start_angles, end_angles, steps)

# Command the arm to follow the path
command_arm_to_follow_path(robot, path)

# Evaluate the error
error_path = evaluate_path_following_error(path, robot)

In [ ]:
def home_to_cupstack():
    Arm.Arm_serial_servo_write6(angle-90, angle, angle, angle, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle, angle, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle-90, angle+45, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle-90, angle+45, angle, angle+90, s_time) #gripper closing 
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-30, angle-75, angle+25, angle, angle+90, s_time) #lifting cup straight up
    time.sleep(1)   

In [ ]:
def cupstack_to_dispenser():
    # Define target position
    target_vector = [1, 1, -3] # Desired position of the gripper relative to the base frame, which is at [0, 0, 0]. This is the location where the gripper will end at and the joints will move accordingly. 
    frame_target = np.eye(4) # 4x4 matrix for Rot, Pos and an extra dimension for homogeneous coordinates
    frame_target[:3, 3] = target_vector
    position = frame_target[:3, 3]
    #do we need a line for defining the rotations of target frame corresponding to the #origin_orientation of the gripper? 
    
    #function within ikpy that takes the setup chain and does the math for ik to reach specified target
    ik = dof_chain.inverse_kinematics(position) # Compute the inverse kinematics

    # Translate IK angles to servo commands
    servo_angles = np.degrees(ik) # Convert radians to degrees
    print(servo_angles)
    Arm.Arm_serial_servo_write(1, servo_angles[0], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(2, servo_angles[1], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(3, servo_angles[2], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(4, servo_angles[3], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(5, servo_angles[4], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(6, 90, s_time)
#     Arm.Arm_serial_servo_write6(*servo_angles, s_time) # '*' because servo_angles holds a list of angles when storing convert_to_servo_commands(ik)
    
#     time.sleep(5) #five sec delay to allow for whole arm to move  

#     #Release cup
#     Arm.Arm_serial_servo_write(gripper, 0, s_time)
#     time.sleep(1)  
    

In [ ]:
def main():
    home() #add inherit delay to function 

try :
    main()
except KeyboardInterrupt:
     print(" Program closed! ")
     pass